In [1]:
import os
import cv2
import numpy as np
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.utils import to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [6]:
def load_data(folder, target_size=(224, 224)):
    data = []
    labels = []
    for label in os.listdir(folder):
        label_folder = os.path.join(folder, label)
        if os.path.isdir(label_folder):
            print(f"Loading images from folder: {label_folder}")
            for filename in os.listdir(label_folder):
                if filename.endswith(".jpg"):
                    img = cv2.imread(os.path.join(label_folder, filename))
                    if img is not None:
                        img = cv2.resize(img, target_size)
                        data.append(img)
                        labels.append(label)
                    else:
                        print(f"Unable to load image: {os.path.join(label_folder, filename)}")
    return np.array(data), np.array(labels)

# Load the collected data
data, labels = load_data(r'C:\Users\vanga\Downloads\Data_Num\Data_Num')

Loading images from folder: C:\Users\vanga\Downloads\Data_Num\Data_Num\1
Loading images from folder: C:\Users\vanga\Downloads\Data_Num\Data_Num\2
Loading images from folder: C:\Users\vanga\Downloads\Data_Num\Data_Num\3
Loading images from folder: C:\Users\vanga\Downloads\Data_Num\Data_Num\4
Loading images from folder: C:\Users\vanga\Downloads\Data_Num\Data_Num\5
Loading images from folder: C:\Users\vanga\Downloads\Data_Num\Data_Num\6
Loading images from folder: C:\Users\vanga\Downloads\Data_Num\Data_Num\7
Loading images from folder: C:\Users\vanga\Downloads\Data_Num\Data_Num\8
Loading images from folder: C:\Users\vanga\Downloads\Data_Num\Data_Num\9


In [7]:
print("Number of images loaded:", len(data))

Number of images loaded: 1800


In [8]:
from sklearn.preprocessing import LabelEncoder

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

print("Labels:", label_encoder.classes_)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels_encoded, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print("Training set size:", len(X_train))
print("Testing set size:", len(X_test))
print("Validation set size:", len(X_val))

Labels: ['1' '2' '3' '4' '5' '6' '7' '8' '9']
Training set size: 1152
Testing set size: 360
Validation set size: 288


In [10]:
import os 
os.environ["KERAS_BACKEND"] = "jax" 
import keras

In [13]:
import jax
import jax.numpy as jnp
from flax import linen as nn

In [16]:
# Build CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    #layers.Dropout(0.5),  # Add dropout to the fully connected layer
    layers.Dense(512, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

C:\Users\vanga\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Define the data augmentation settings
datagen = ImageDataGenerator(
    rotation_range=15, 
    width_shift_range=0.1,  
    height_shift_range=0.1,  
    shear_range=0.1, 
    zoom_range=0.1,  
    horizontal_flip=True,  
    vertical_flip=True, 
    fill_mode='nearest' 
)

In [18]:
epochs = 5
history = model.fit(datagen.flow(X_train, tf.keras.utils.to_categorical(y_train, len(label_encoder.classes_)), batch_size=64),epochs=epochs, steps_per_epoch=len(X_train) // 32, validation_data=(X_test, tf.keras.utils.to_categorical(y_test, len(label_encoder.classes_))),validation_steps=len(X_test) // 32)

Epoch 1/5


C:\Users\vanga\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


18/36 ━━━━━━━━━━━━━━━━━━━━ 1:06 4s/step - accuracy: 0.1633 - loss: 477.6761

C:\Users\vanga\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


36/36 ━━━━━━━━━━━━━━━━━━━━ 108s 2s/step - accuracy: 0.2084 - loss: 344.4494 - val_accuracy: 0.4750 - val_loss: 1.5218
Epoch 2/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step - accuracy: 0.4816 - loss: 1.5364 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 0.6148 - loss: 1.1561 - val_accuracy: 0.8139 - val_loss: 0.6334
Epoch 4/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 74s 2s/step - accuracy: 0.7697 - loss: 0.7353 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.8033 - loss: 0.5941 - val_accuracy: 0.9556 - val_loss: 0.1275


In [19]:
test_loss, test_accuracy = model.evaluate(X_test, tf.keras.utils.to_categorical(y_test, len(label_encoder.classes_)))
print(f'Test accuracy: {test_accuracy:.4f}')

12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 422ms/step - accuracy: 0.9739 - loss: 0.1154
Test accuracy: 0.9556


In [22]:
num_classes = len(labels)
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
y_val = to_categorical(y_val,num_classes)
print(num_classes)
print(y_train)

1800
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


In [23]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the first 100 layers and fine-tune the rest
for layer in base_model.layers[:100]:
    layer.trainable = False
for layer in base_model.layers[100:]:
    layer.trainable = True

# Add custom head to the model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)

# Output layer with softmax activation for classification
predictions = Dense(num_classes, activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
from keras.callbacks import EarlyStopping

# Train the model with data augmentation and early stopping
history_resnet50 = model.fit(datagen.flow(X_train, y_train, batch_size=32), validation_data=(X_val, y_val), epochs=5, callbacks=[EarlyStopping(monitor='val_loss', patience=3)])

Epoch 1/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 247s 5s/step - accuracy: 0.3858 - loss: 5.1126 - val_accuracy: 0.4965 - val_loss: 2.7306
Epoch 2/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 237s 7s/step - accuracy: 0.9909 - loss: 0.1242 - val_accuracy: 0.9896 - val_loss: 0.5187
Epoch 3/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 187s 5s/step - accuracy: 1.0000 - loss: 0.0130 - val_accuracy: 1.0000 - val_loss: 0.0821
Epoch 4/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 185s 5s/step - accuracy: 0.9973 - loss: 0.0127 - val_accuracy: 1.0000 - val_loss: 0.0199
Epoch 5/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 176s 5s/step - accuracy: 1.0000 - loss: 0.0096 - val_accuracy: 1.0000 - val_loss: 0.0077


In [25]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_accuracy}')

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=2) 

print(f'validation accuracy: {val_accuracy:.4f}')

12/12 - 25s - 2s/step - accuracy: 1.0000 - loss: 0.0076
Test accuracy: 1.0
9/9 - 17s - 2s/step - accuracy: 1.0000 - loss: 0.0077
validation accuracy: 1.0000


In [26]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Flatten

In [27]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers for classification
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


history_vgg = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32)

Epoch 1/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 245s 7s/step - accuracy: 0.8334 - loss: 5.1757 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 317s 9s/step - accuracy: 0.9987 - loss: 0.0050 - val_accuracy: 1.0000 - val_loss: 1.1176e-08
Epoch 3/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 309s 9s/step - accuracy: 1.0000 - loss: 6.2269e-07 - val_accuracy: 0.9965 - val_loss: 0.0030
Epoch 4/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 312s 9s/step - accuracy: 1.0000 - loss: 3.9171e-04 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 328s 9s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


In [28]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)

12/12 ━━━━━━━━━━━━━━━━━━━━ 84s 7s/step - accuracy: 1.0000 - loss: 0.0000e+00
Test Accuracy: 100.00%
9/9 ━━━━━━━━━━━━━━━━━━━━ 66s 7s/step - accuracy: 1.0000 - loss: 0.0000e+00
